In [2]:
import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
def runLR(xtrain, ytrain, xtest, ytest):
    lr = LogisticRegression()
    lr.fit(xtrain, ytrain)
    print('Accuracy: ', lr.score(xtest, ytest))
    return lr

In [4]:
def runDTC(xtrain, ytrain, xtest, ytest, names=None, f=30):
    dt = DecisionTreeClassifier()
    dt.fit(xtrain, ytrain)
    print('Accuracy: ', dt.score(xtest, ytest))
    print('Top {} features using DTC'.format(f))
    if names is not None:
        if type(names) is list:
            print([names[z] for z in np.argsort(dt.feature_importances_)[::-1][:f]])
        else:
            print(names[np.argsort(dt.feature_importances_)[::-1][:f]])
    else:
        print([np.argsort(dt.feature_importances_)[::-1][:f]])
    return dt

# 20 News Groups

In [5]:
ngtrain = fetch_20newsgroups(subset='train', data_home='../Data')
ngtest = fetch_20newsgroups(subset='test', data_home='../Data')

In [6]:
ngtrain_vectorizer = TfidfVectorizer(stop_words='english', strip_accents='ascii')
_ngtrain_vectors = ngtrain_vectorizer.fit_transform(ngtrain.data)
ngtest_vectorizer = TfidfVectorizer(stop_words='english', strip_accents='ascii')
_ngtest_vectors = ngtest_vectorizer.fit_transform(ngtest.data)

In [7]:
# remove features in test that are not found in train
get_indices = lambda x, y: list(set(np.searchsorted(x, y)))
common_features = np.intersect1d(ngtrain_vectorizer.get_feature_names(), ngtest_vectorizer.get_feature_names())
ngtrain_vectors = _ngtrain_vectors[:, get_indices(ngtrain_vectorizer.get_feature_names(), common_features)]
ngtest_vectors = _ngtest_vectors[:, get_indices(ngtest_vectorizer.get_feature_names(), common_features)]

## Logistic Regression

In [8]:
nglr = runLR(ngtrain_vectors, ngtrain.target, ngtest_vectors, ngtest.target)

Accuracy:  0.827535847053


LogisticRegression of sklearn package uses the one-vs-rest model for a multi-class classification. Because of this, we get 20 (no. of classes in data) different sets of coefficients. Now in order to pick the best 30 features, I have chosen to pick the top 30 from each class instead of choosing the top 30 over all the classes. 

Since we are using the one-vs-rest model, to predict the output probability for each class, we can choose the most important features (30 in this eg) and use them to classify. 

In [9]:
f = 30
best_features = {}
for i in range(nglr.coef_.shape[0]):
    print('Top {} features for class `{}`:'.format(f, ngtrain.target_names[i]))
    _features = common_features[np.argsort(nglr.coef_[i])[::-1][:f]]
    best_features.update({ngtrain.target_names[i]: _features})
    print(_features)
    print('--'*15)

Top 30 features for class `alt.atheism`:
['keith' 'atheism' 'atheists' 'islamic' 'caltech' 'okcforum' 'god' 'islam'
 'mathew' 'jaeger' 'livesey' 'rushdie' 'mangoe' 'benedikt' 'umd' 'religion'
 'osrhe' 'wingate' 'edu' 'kmr4' 'sgi' 'bible' 'cobb' 'mantis' 'gregg' 'tek'
 'solntze' 'atheist' 'bobby' 'ico']
------------------------------
Top 30 features for class `comp.graphics`:
['graphics' 'image' '3d' 'polygon' 'tiff' 'images' 'files' 'pov' '3do'
 'format' 'points' 'animation' 'package' 'gif' 'vga' 'sphere' 'color'
 'vesa' 'surface' 'algorithm' 'program' 'file' '42' 'looking' 'library'
 'code' 'software' 'quicktime' 'xv' 'mpeg']
------------------------------
Top 30 features for class `comp.os.ms-windows.misc`:
['windows' 'file' 'ax' 'driver' 'drivers' 'files' 'cica' 'dos' 'win' 'nt'
 'risc' 'ini' 'mouse' 'ms' 'win3' 'fonts' 'font' 'ftp' 'printer' 'access'
 'microsoft' 'using' 'bmp' 'tw' 'program' 'card' 'manager' 'diamond'
 'norton' 'deskjet']
------------------------------
Top 30 featu

## Decision Tree Classifier

In [42]:
ngdt = runDTC(ngtrain_vectors, ngtrain.target, ngtest_vectors, ngtest.target, common_features)

Accuracy:  0.57063197026
Top 30 features using DTC
['clipper' 'dod' 'sale' 'windows' 'hockey' 'god' 'space' 'israel' 'car'
 'gun' 'baseball' 'turkish' 'bike' 'mac' 'graphics' 'window' 'nhl'
 'rutgers' 'geb' 'atheists' 'encryption' 'motif' 'article' 'orbit'
 'controller' 'team' 'nntp' 'com' 'edu' 'apple']


# MNIST

In [61]:
mnist = fetch_mldata('MNIST original', data_home='../Data')
mxtrain, mxtest, mytrain, mytest = train_test_split(mnist.data, mnist.target, test_size=0.33, random_state=42)

## Logistic Regression

In [64]:
mnlr = runLR(mxtrain, mytrain, mxtest, mytest)

Accuracy:  0.915714285714


In [66]:
mnlr.coef_.shape

(10, 784)

In [71]:
f = 30
for i in range(mnlr.coef_.shape[0]):
    print('Top {} features for class `{}`:'.format(f, np.unique(mnist.target)[i]))
    print(np.argsort(mnlr.coef_[i])[::-1][:f])
    print('--'*15)

Top 30 features for class `0.0`:
[224 503 614  44  89  45 226  69 676 109 668 311  87 143 172 641 118  71
 339 586 201 639  59 717 360 711 694 591 257 370]
------------------------------
Top 30 features for class `1.0`:
[641 359 307 312 256 472 697 693 418 416 281 395 586 670 583 174 143  67
 396 453 114 276 342 120 531 203 258 535 446 698]
------------------------------
Top 30 features for class `2.0`:
[503 391 706 137 478 505 367 198 277 423 696  80 418 340 221  79 422  34
  33 333 165 614 390 396 143 474 313 585 562  14]
------------------------------
Top 30 features for class `3.0`:
[702 197  89 446 361 449 393 586 421 450 561 646 136 668 675 695 387 670
 108 590 104 338 367 451 722 395 472 705 748 676]
------------------------------
Top 30 features for class `4.0`:
[478 422 393 697 475 337  90 642 281 670 446 558 723 751 109 227 583 254
 117 613 360 311 136 556 417 107 669 119 105  68]
------------------------------
Top 30 features for class `5.0`:
[ 88 390 394 361 362 392 724 734

## Decision Tree

In [45]:
mndtc = runDTC(mxtrain, mytrain, mxtest, mytest)

Accuracy:  0.868354978355
Top 30 features using DTC
[array([489, 350, 435, 347, 542, 211, 432, 270, 597, 486, 155, 354, 273,
       550, 296, 290, 405, 656, 657, 352, 297, 596, 458, 658, 301, 380,
       234, 404, 598, 515])]


# Spambase dataset

In [46]:
def spambasedata():
    f = open('../Data/spambase.data')
    target = []
    spam_data = []    
    for line in f.readlines():
        target.append(int(line[-2]))
        line = line[:-3].strip().split(',')
        spam_data.append(list(map(float, line)))
    target = np.array(target)
    f.close()
    f = open('../Data/spambase.names')
    names = []
    for line in f.readlines()[33:]:
        names.append(line.strip().split(':')[0])
    f.close()
    return spam_data, target, names

sdata, starget, snames = spambasedata()

In [47]:
sxtrain, sxtest, sytrain, sytest = train_test_split(sdata, starget, test_size=0.33, random_state=42)

## Logistic Regression

In [48]:
slr = runLR(sxtrain, sytrain, sxtest, sytest)

Accuracy:  0.930875576037


In [67]:
f = 30
print('Top {} features:'.format(f))
print([snames[j] for j in np.argsort(slr.coef_[0])[::-1][:f]])

Top 30 features:
['char_freq_$', 'word_freq_000', 'word_freq_remove', 'word_freq_addresses', 'word_freq_order', 'char_freq_#', 'word_freq_free', 'word_freq_business', 'word_freq_technology', 'word_freq_credit', 'word_freq_3d', 'word_freq_our', 'word_freq_over', 'word_freq_internet', 'word_freq_650', 'word_freq_money', 'word_freq_your', 'word_freq_font', 'char_freq_!', 'word_freq_report', 'word_freq_415', 'word_freq_all', 'word_freq_you', 'word_freq_email', 'word_freq_mail', 'word_freq_people', 'capital_run_length_longest', 'capital_run_length_total', 'capital_run_length_average', 'word_freq_telnet']


## Decision Tree classifier

In [60]:
sdt = runDTC(sxtrain, sytrain, sxtest, sytest, snames)

Accuracy:  0.909809084924
Top 30 features using DTC
['char_freq_$', 'word_freq_remove', 'char_freq_!', 'word_freq_hp', 'capital_run_length_average', 'word_freq_free', 'capital_run_length_longest', 'word_freq_our', 'word_freq_you', 'word_freq_edu', 'word_freq_your', 'word_freq_george', 'word_freq_will', 'word_freq_internet', 'word_freq_85', 'word_freq_technology', 'word_freq_all', 'word_freq_order', 'word_freq_re', 'word_freq_business', 'word_freq_pm', 'capital_run_length_total', 'word_freq_hpl', 'char_freq_;', 'word_freq_money', 'char_freq_#', 'word_freq_make', 'char_freq_(', 'word_freq_receive', 'word_freq_mail']
